In [1]:
import yaml
import collections

import pandas as pd

In [2]:
def read_init_file(init_file_name):
    """Reads in the model specification from yaml file"""

    # Import yaml initialization file as dictionary init_dict
    if isinstance(init_file_name, str):
        with open(init_file_name) as y:
            init_dict = yaml.load(y, Loader=yaml.FullLoader)
    else:
        init_dict = init_file_name

    init_dict = expand_init_dict(init_dict)

    model_params = create_namedtuple(init_dict)

    return model_params


def flatten_init_dict(init_dict):
    """Removes the grouping from the nested dictionary"""

    groups = [
        "GENERAL",
        "CONSTANTS",
        "INITIAL_CONDITIONS",
        "SIMULATION",
        "SOLUTION",
    ]

    model_spec = dict()

    for group in groups:

        keys_ = list(init_dict[group].keys())
        values_ = list(init_dict[group].values())

        for k_, key_ in enumerate(keys_):

            model_spec[key_] = values_[k_]

    return model_spec


def dict_to_namedtuple(dictionary):
    """Coverts non-nested dictionary to namedtuple"""

    return collections.namedtuple("model_specification", dictionary.keys())(**dictionary)


Import yaml file as init dict
--------------------------------

In [3]:
# Import yaml initialization file as dictionary init_dict

with open('sim_toy_model_init_file_1000_estimagic.yml') as y:
    init_dict = yaml.load(y, Loader=yaml.FullLoader)

In [4]:
init_dict

{'GENERAL': {'num_periods': 30},
 'CONSTANTS': {'delta': 0.98, 'mu': -0.56, 'benefits': 4.5},
 'INITIAL_CONDITIONS': {'educ_max': 16, 'educ_min': 10},
 'SIMULATION': {'seed_sim': 102, 'num_agents_sim': 1000},
 'SOLUTION': {'seed_emax': 635, 'num_draws_emax': 500},
 'PARAMETERS': {'const_wage_eq': {'gamma_0s1': [1.792, 'False', 1.0, 3.0],
   'gamma_0s2': [1.808, 'False', 1.0, 3.0],
   'gamma_0s3': [1.856, 'False', 1.0, 3.0]},
  'exp_returns': {'gamma_1s1': [0.122, 'False', 0.05, 0.4],
   'gamma_1s2': [0.199, 'False', 0.05, 0.4],
   'gamma_1s3': [0.266, 'False', 0.05, 0.4]},
  'exp_accm': {'g_s1': [0.2, 'False', 0.005, 0.6],
   'g_s2': [0.2, 'False', 0.005, 0.6],
   'g_s3': [0.2, 'False', 0.005, 0.6]},
  'exp_deprec': {'delta_s1': [0.081, 'False', 0.001, 0.15],
   'delta_s2': [0.057, 'False', 0.001, 0.15],
   'delta_s3': [0.073, 'False', 0.001, 0.15]},
  'diutil_work': {'const_p': [1.88, 'False', 1.0, 4.0],
   'const_f': [2.33, 'False', 1.0, 4.0]},
  'hetrg_unobs': {'theta_p1': [-0.2, 'F

Extract model specification parameters as namedtuple
-----------------------------------------------------------------

In [5]:
model_spec = flatten_init_dict(init_dict)

In [6]:
model_spec = dict_to_namedtuple(model_spec)

In [7]:
model_spec

model_specification(num_periods=30, delta=0.98, mu=-0.56, benefits=4.5, educ_max=16, educ_min=10, seed_sim=102, num_agents_sim=1000, seed_emax=635, num_draws_emax=500)

Extract estimation parameters as multiindex data frame
------------------------------------------------------------------

In [8]:
estim_paras = init_dict['PARAMETERS']

In [9]:
estim_paras

{'const_wage_eq': {'gamma_0s1': [1.792, 'False', 1.0, 3.0],
  'gamma_0s2': [1.808, 'False', 1.0, 3.0],
  'gamma_0s3': [1.856, 'False', 1.0, 3.0]},
 'exp_returns': {'gamma_1s1': [0.122, 'False', 0.05, 0.4],
  'gamma_1s2': [0.199, 'False', 0.05, 0.4],
  'gamma_1s3': [0.266, 'False', 0.05, 0.4]},
 'exp_accm': {'g_s1': [0.2, 'False', 0.005, 0.6],
  'g_s2': [0.2, 'False', 0.005, 0.6],
  'g_s3': [0.2, 'False', 0.005, 0.6]},
 'exp_deprec': {'delta_s1': [0.081, 'False', 0.001, 0.15],
  'delta_s2': [0.057, 'False', 0.001, 0.15],
  'delta_s3': [0.073, 'False', 0.001, 0.15]},
 'diutil_work': {'const_p': [1.88, 'False', 1.0, 4.0],
  'const_f': [2.33, 'False', 1.0, 4.0]},
 'hetrg_unobs': {'theta_p1': [-0.2, 'False', -0.4, -0.05],
  'theta_f1': [-0.5, 'False', -0.8, -0.15],
  'share_1': [0.5, 'False', 0.001, 0.999]},
 'sd_wage_shock': {'sigma_1': [0.01, 'False', 0.001, 0.8],
  'sigma_2': [0.2, 'False', 0.001, 0.8],
  'sigma_3': [0.4, 'False', 0.001, 0.8]}}

In [10]:
def transform_estim_paras_dict(estim_paras):
    """Transforms nested dictionary of parameters to estimate
    into a dictionary with double value key
    corresponding to category and name."""
    
    estim_paras_dict_transformed = {(outerKey, innerKey): values for outerKey, innerDict in estim_paras.items() for innerKey, values in innerDict.items()}
    
    return estim_paras_dict_transformed

In [11]:
estim_paras_dict_transformed = transform_estim_paras_dict(estim_paras)
estim_paras_dict_transformed

{('const_wage_eq', 'gamma_0s1'): [1.792, 'False', 1.0, 3.0],
 ('const_wage_eq', 'gamma_0s2'): [1.808, 'False', 1.0, 3.0],
 ('const_wage_eq', 'gamma_0s3'): [1.856, 'False', 1.0, 3.0],
 ('exp_returns', 'gamma_1s1'): [0.122, 'False', 0.05, 0.4],
 ('exp_returns', 'gamma_1s2'): [0.199, 'False', 0.05, 0.4],
 ('exp_returns', 'gamma_1s3'): [0.266, 'False', 0.05, 0.4],
 ('exp_accm', 'g_s1'): [0.2, 'False', 0.005, 0.6],
 ('exp_accm', 'g_s2'): [0.2, 'False', 0.005, 0.6],
 ('exp_accm', 'g_s3'): [0.2, 'False', 0.005, 0.6],
 ('exp_deprec', 'delta_s1'): [0.081, 'False', 0.001, 0.15],
 ('exp_deprec', 'delta_s2'): [0.057, 'False', 0.001, 0.15],
 ('exp_deprec', 'delta_s3'): [0.073, 'False', 0.001, 0.15],
 ('diutil_work', 'const_p'): [1.88, 'False', 1.0, 4.0],
 ('diutil_work', 'const_f'): [2.33, 'False', 1.0, 4.0],
 ('hetrg_unobs', 'theta_p1'): [-0.2, 'False', -0.4, -0.05],
 ('hetrg_unobs', 'theta_f1'): [-0.5, 'False', -0.8, -0.15],
 ('hetrg_unobs', 'share_1'): [0.5, 'False', 0.001, 0.999],
 ('sd_wage_sh

In [12]:
estim_paras_df = pd.DataFrame(estim_paras_dict_transformed, index = ['value', 'fixed', 'lower', 'upper']).T

In [13]:
estim_paras_df

value  fixed  lower  upper
const_wage_eq gamma_0s1  1.792  False      1      3
              gamma_0s2  1.808  False      1      3
              gamma_0s3  1.856  False      1      3
exp_returns   gamma_1s1  0.122  False   0.05    0.4
              gamma_1s2  0.199  False   0.05    0.4
              gamma_1s3  0.266  False   0.05    0.4
exp_accm      g_s1         0.2  False  0.005    0.6
              g_s2         0.2  False  0.005    0.6
              g_s3         0.2  False  0.005    0.6
exp_deprec    delta_s1   0.081  False  0.001   0.15
              delta_s2   0.057  False  0.001   0.15
              delta_s3   0.073  False  0.001   0.15
diutil_work   const_p     1.88  False      1      4
              const_f     2.33  False      1      4
hetrg_unobs   theta_p1    -0.2  False   -0.4  -0.05
              theta_f1    -0.5  False   -0.8  -0.15
              share_1      0.5  False  0.001  0.999
sd_wage_shock sigma_1     0.01  False  0.001    0.8
              sigma_2      0.2  False  0.001    0.8
              sigma_3      0.4  False  0.001    0.8

Example usage in functions
--------------------------------

In [14]:
import numpy as np

In [15]:
# Set example values
states = [1, 12, 2, 0, 1, 1]
covariates = 12

In [18]:
def calculate_log_wage_systematic(model_spec, estim_paras_df, states, covariates):
    """Calculate systematic wages, i.e., wages net of shock, for all states."""

    exp_p, exp_f = states[3], states[4]
    educ_level = covariates - model_spec.educ_min

    # Construct wage components
    gamma_0s = estim_paras_df.loc[('const_wage_eq'),'value'].to_numpy()[educ_level]
    gamma_1s = estim_paras_df.loc[('const_wage_eq'),'value'].to_numpy()[educ_level]
    period_exp_sum = exp_p * estim_paras_df.loc[('exp_accm'),'value'].to_numpy()[educ_level] + exp_f
    depreciation = 1 - estim_paras_df.loc[('exp_deprec'),'value'].to_numpy()[educ_level]

    # Calculate wage in the given state
    period_exp_total = period_exp_sum * depreciation + 1
    returns_to_exp = gamma_1s * np.log(period_exp_total)
    log_wage_systematic = gamma_0s + returns_to_exp

    return log_wage_systematic

In [20]:
test = calculate_log_wage_systematic(model_spec, estim_paras_df, states, covariates)
test

3.073469906786218